In [16]:
const http = require('http');
const url = require('url');
const querystring = require('querystring');
const jade = require('jade');

In [ ]:
const routers = {
    ['GET /']() {
      let result = {
        'json': true,
        'headers': {    // 设置额外的http头参数
          'auth': 'alvin'
        }
      };

  let message = 'Hello node.js';
  // 判断请求参数对象中是否有name属性
  if (context.parameters.name) {
    message += ', have fun ' + context.parameters.name;
  }
  // 为结果对象设置content属性
  result.content = {
    'status': 'success',
    'message': message
  };
  return result;
},

// 处理'html'类型的请求
'GET /new': function () {
  return {
    'html': true,
    'parameters': {
      'name': 'Alvin'
    }
  };
},

// 处理'/new'路径POST方法请求
'POST /': function (context) {
  let name = context.parameters.name;
  let password = context.parameters.password;

  if (!name || !password) {
    let errors = {};
    if (!name) {
      errors['name'] = '帐号不能为空';
    }
    if (!password) {
      errors['password'] = '密码不能为空';
    }
    return {
      'html': true,
      'view': 'GET_new',
      'parameters': {
        'errors': errors,
        'name': name,
        'password': password
      }
    };
  }
  return {
    'redirect': true,
    'url': '/'
  }
},

// 处理'/redirect'路径GET方法请求
'GET /redirect': function (context) {
  let url = '/';
  if (context.parameters.url) {
    url = context.parameters.url;
  }
  return {
    'redirect': true,
    'url': url
  };
},

// 处理404错误
'404': function () {
  return {
    'json': true,
    'content': {
      'status': 'error',
      'message': 'Resource not found.'
    }
  };
}
};

In [19]:
function httpService(request, response) {    
    console.log(`* request version: ${request.httpVersion}`);
    console.log(`* request header is: ${request.headers.host}`);

    const uri = url.parse(request.url, true, false);
    const context = {
        headers: request.headers, 
        parameters: {
            ...uri.query
        }
    };

    const chunks = [];
    request.on('data', chunk => chunks.push(chunk));
    request.on('end', () => {
    if (chunks.length > 0) {
        const body = chunks.join('');
        console.log(`* request body is: ${body}`);

        context.parameters = {
            ...querystring.parse(body),
            ...context.parameters
        }
    }

    function execute(code, router) {
        if (router) {
            const result = (() => {
                if (typeof(router) === 'function') {
                    return router(context);
                }
                return router.controller(context);
            })();

            if (result.redirect) {
                response.writeHeader(302, { 'Location': result.url });
                response.end();
                return;
            }
            
            if (result.json) {
                const headers = {
                    'Content-Type': 'application/json',
                    ...result.headers
                };
                response.writeHeader(code, headers);
                response.end(JSON.stringify(result.content));
                return;
            }
            
            if (result.html) {
                const headers = {
                    'Content-Type': 'text/html',
                    ...result.headers
                };
                response.writeHeader(code, headers);
            
                const view = `view/${(result.view || (request.method + '_' + uri.pathname.substr(1)))}.jade`;
                jade.renderFile(url, { errors: {}, ...result.parameters }, (err, html) => {
                    if (err) {
                        throw err;
                    }
                    response.end(html);
                });
                return;
            }
            
            response.writeHeader(500, { 'Content-Type': 'text/html' });
            response.end();
        } else {
            response.writeHeader(404, { 'Content-Type': 'text/html' });
            response.end();
        }
    }
        
    let router = routers[request.method + ' ' + (uri.pathname || '/')];
    try {
    if (router) {
      // 找到指定的router对象, 进行处理
      execute(200, router);
    } else {
      // 未找到指定的router对象, 尝试
      execute(404, routers['404']);
    }
    } catch (e) {
    console.log(e);
    execute(500, routers['500']);
    }
    });
}

76:18 - Cannot find name 'routers'. Did you mean 'router'?
83:20 - Cannot find name 'routers'. Did you mean 'router'?
87:18 - Cannot find name 'routers'. Did you mean 'router'?
